In [1]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-
"""
Run a YOLO_v3 style detection model on test images.
"""

import colorsys
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from keras.utils import multi_gpu_model
gpu_num=1

#- Added
import cv2
cap = cv2.VideoCapture(0)
camera_scale = 1.
#-

import csv
import glob 
import re

Using TensorFlow backend.


In [2]:
dir_name = "img_data"

# color range
lower_red = [[140,20,20],[0,20,20]]
upper_red = [[180,255,255],[10,255,255]]

lower_green = [35, 20, 20]
upper_green = [95, 255, 255]

# setting the threshold
threshold = 60

In [9]:
class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/yolo.h5' # model path or trained weights path
        self.anchors_path = 'model_data/yolo_anchors.txt'
        self.classes_path = 'model_data/coco_classes.txt'
        self.score = 0.3
        self.iou = 0.45
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.model_image_size = (416, 416) # fixed size or (None, None), hw
        self.boxes, self.scores, self.classes = self.generate()
        self.count = 0

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

#         print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

#         print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

#         with open('rectangle.csv', 'w') as f:
#             writer = csv.writer(f)
        #count = 0
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            if predicted_class == 'traffic light':
                label = '{} {:.2f}'.format(predicted_class, score)
                draw = ImageDraw.Draw(image)
                label_size = draw.textsize(label, font)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
                print(label, (left, top), (right, bottom))
#                     writer.writerow([top, bottom, left, right])
                cropped_img = image.crop((left, top, right, bottom))
                cropped_img.save(f"img_data/rectangle/{self.count}.jpg")
                #color_prediction
                
                color_extraction(dir_name, 'rectangle', 'red', lower_red, upper_red)
                color_extraction(dir_name, 'rectangle', 'green', lower_green, upper_green)

                red_raio = color_ratio(dir_name, "red", threshold)
                green_ratio = color_ratio(dir_name, "green", threshold)
                color_predict(red_raio, green_ratio, self.count)
                self.count += 1

                if top - label_size[1] >= 0:
                    text_origin = np.array([left, top - label_size[1]])
                else:
                    text_origin = np.array([left, top + 1])

                # My kingdom for a good redistributable image drawing library.
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=self.colors[c])
                draw.rectangle(
                    [tuple(text_origin), tuple(text_origin + label_size)],
                    fill=self.colors[c])
                draw.text(text_origin, label, fill=(0, 0, 0), font=font)
                del draw

                end = timer()
                print(end - start)
        return image

    def close_session(self):
        self.sess.close()

In [10]:
def detect_img(yolo):
    while True:
        ret, image = cap.read()
        if cv2.waitKey(10) == 27:
            break
        h, w = image.shape[:2]
        rh = int(h * camera_scale)
        rw = int(w * camera_scale)
        image = cv2.resize(image, (rw, rh))
        image = image[:,:,(2,1,0)]
        image = Image.fromarray(image)
        r_image = yolo.detect_image(image)
        out_img = np.array(r_image)[:,:,(2,1,0)]
        cv2.imshow("YOLOv2", np.array(out_img))
        cv2.waitKey(0)
    yolo.close_session()

def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def get_img_path(dir_name:str, version:str):
    files = f"{dir_name}/{version}/"
    img_path = sorted(glob.glob(str(files+"*.jpg")), key=numericalSort)
    return img_path           

#Phase2:color extracting
#create color mask
def create_mask(color:str, lower_color:list, upper_color:list, img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    if color == 'red':
        lower_red1 = np.array(lower_color[0])
        upper_red1 = np.array(upper_color[0])
        lower_red2 = np.array(lower_color[1])
        upper_red2 = np.array(upper_color[1])
        red_mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
        red_mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask = red_mask1 + red_mask2
    else:
        lower_color = np.array(lower_color)
        upper_color = np.array(upper_color)
        mask = cv2.inRange(hsv, lower_color, upper_color)
    return mask

#create pics extracted the paricular color
def color_extraction(dir_name:str, version:str, color:str, lower_color:list, upper_color:list):
    img_path = get_img_path(dir_name, version)
    for i in range(len(img_path)):
        img = cv2.imread(f"{img_path[i]}")
        mask = create_mask(color, lower_color, upper_color, img)
        target = cv2.bitwise_and(img, img, mask=mask)
        cv2.imwrite(f"{dir_name}/{color}/{i}.jpg", target)
        

##Phase3:predict
#calucurate the ratio of color in the pics
def color_ratio(dir_name:str, color:str, threshold:int):
    color_ratio = np.zeros(len(get_img_path(dir_name, color)))
    for i in range(len(get_img_path(dir_name, color))):
        img = cv2.imread(f"{dir_name}/{color}/{i}.jpg", 0)
        ret, img_thresh = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
#         cv2.imwrite(f"{dir_name}/binary_{color}/{i}.jpg", img_thresh)
        colorPixels = cv2.countNonZero(img_thresh)
        wholeSize = img.size
        color_ratio[i] += colorPixels/wholeSize*100
    return color_ratio

#judge the color of sign
def color_predict(red:list, green:list, num:int):
    if red[num]>green[num]:
        print('The color is Red. Please stop.')
    else:
        print('The color is blue. Please cross the street carefully')

In [ ]:
if __name__ == '__main__':
    detect_img(YOLO())

model_data/yolo.h5 model, anchors, and classes loaded.
<class 'numpy.ndarray'>
traffic light 0.37 (3, 0) (449, 480)
The color is blue. Please cross the street carefully
2.6101449750130996
traffic light 0.68 (62, 315) (377, 460)
The color is blue. Please cross the street carefully
2.6255307010142133
<class 'numpy.ndarray'>
traffic light 0.30 (15, 6) (451, 480)
The color is blue. Please cross the street carefully
0.06341623200569302
traffic light 0.73 (75, 323) (387, 463)
The color is blue. Please cross the street carefully
0.08889478899072856
<class 'numpy.ndarray'>
traffic light 0.72 (94, 320) (419, 462)
The color is blue. Please cross the street carefully
0.07217674993444234
<class 'numpy.ndarray'>
traffic light 0.77 (99, 330) (419, 467)
The color is blue. Please cross the street carefully
0.06507363496348262
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'nu